# Market selection by using portfolio theory

In [1]:

import os,sys,matplotlib,pandas,numpy,scipy,dateutil
import pandas as pd
import numpy as np
from pylab import *
from datetime import datetime, timedelta
import time
from statsmodels.distributions.empirical_distribution import ECDF
import sqlite3
import itertools, random
import pickle
conn = None


## Reading price data.
Connect to database first, and the slurp the data for the given markets 

In [2]:
markets = [ {'type': 'g2.2xlarge', 'region': 'us-east-1', 'AZ': 'a', 'OS': 'Linux'} ,
                     {'type': 'c3.xlarge', 'region': 'us-east-1', 'AZ': 'a', 'OS': 'Linux'} ,
                     {'type': 'r3.large', 'region': 'us-east-1', 'AZ': 'a', 'OS': 'Linux'} ,
                     {'type': 'm3.medium', 'region': 'us-east-1', 'AZ': 'a', 'OS': 'Linux'} ,
                     {'type': 'd2.8xlarge', 'region': 'us-east-1', 'AZ': 'a', 'OS': 'Linux'}
                     ]

In [14]:
str(instance_string(markets[0]))

'us-east-1a-Linux-g2.2xlarge'

In [3]:
def init_db_connection() :
    global conn
    conn = sqlite3.connect('/home/prateeks/spot_prices_2015/aws.db')
    return 

init_db_connection();


In [4]:
print conn

In [5]:
def instance_to_MarketId(i) :
    """ The database table names or the file names  """
    osname = {'Linux':'Linux/UNIX'}
    return i['region']+i['AZ']+'-'+osname[i['OS']]+'-'+i['type']


In [13]:
def instance_string(i):
    return i['region']+i['AZ']+'-'+ i['OS']+'-'+i['type']

In [36]:
def read_data_sqlite(instance , mktstring=None, months=['201503','201504','201505','201506','201507','201508'],pickle_data=True):
    global conn
    table_name_base='AKIAJXNAQH5WHJFPNCXA_' # '201503'
    if mktstring == None:
        mktstring = instance_to_MarketId(instance)
    output=[]
    ondem=[]
    ondemtable = 'on_demand_price'
    #Need to remove avail zone from mkt string argh!
   #TODO: function for converting to the on-demand string 
    ondem_mktstring = mktstring.replace('a-','-')
    ondem_mktstring = ondem_mktstring.replace('/UNIX','')
    q2 = 'SELECT SpotPrice FROM {} WHERE MarketId=\'{}\' ;'.format(ondemtable, ondem_mktstring)
    d2 = conn.execute(q2)
    ondem = d2.fetchall()

    for month in months :
        table_name = table_name_base+month
        query='SELECT Timestamp, SpotPrice FROM {} WHERE MarketId=\'{}\' ;'.format(table_name, mktstring)
        #print query
        d = conn.execute(query)
        output.extend(d.fetchall())
    if pickle_data is True :
        fname = instance_string(instance)+'.pickle'
        with open(fname, 'w') as f :
            pickle.dump((instance, output, ondem[0][0]), f)
    return output, ondem[0][0]


In [44]:
def get_spot_data(markets, from_pickle=False):
    """
    markets is a list of dictionaries. Return [(mkt,on-demprice,[(t,spot-price)])]
    """
    global conn
    
    if from_pickle is False:
        data = []
        init_db_connection()
        #c3.xlarge seems volatile
        for i in markets :
            ts, ondem = read_data_sqlite(i)
            spot = [(time.mktime(dateutil.parser.parse(t).timetuple()), float(f)) for (t,f) in ts]
            nspot = np.array(spot)
            data.append((i, float(ondem), nspot))
    else : 
        #Read all the pickle files from the directory and call get_pickled_data
        for file in os.listdir('.'):
            if file.endswith(".pickle") :
                with open(file,'r') as f:
                    (instance, ts, ondem) = get_picked_data(f)
                    spot = [(time.mktime(dateutil.parser.parse(t).timetuple()), float(f)) for (t,f) in ts]
                    nspot = np.array(spot)
                    data.append((i, float(ondem), nspot))

    return data


In [45]:
def get_picked_data(f):
    """ Get pickled data from a file. Return (instance, ts, ondem) tuple. """
    (instance, ts, ondem) = pickle.load(f)
    return (instance, ts, ondem)
    
    

In [46]:
ls

all-cdf.py               time_series_1.ipynb
Bidding_1.ipynb          Untitled0-Copy0_1.ipynb
Bidding_2.ipynb          Untitled0.ipynb
Correlation-Copy0.ipynb  Untitled1.ipynb
Correlation.ipynb        Untitled2.ipynb
ft-mechanisms.py         us-east-1a-Linux-c3.xlarge.pickle
overyears.py             us-east-1a-Linux-d2.8xlarge.pickle
parr_ex_price.py         us-east-1a-Linux-g2.2xlarge.pickle
README.md                us-east-1a-Linux-m3.medium.pickle
spike_correlation.ipynb  us-east-1a-Linux-r3.large.pickle
spot_analysis.py         var-risk-Copy1.ipynb
spotfail.py*             var-risk.ipynb
testr.R                  years.ipynb


In [47]:
data = get_spot_data(markets, from_pickle=True)

UnboundLocalError: local variable 'data' referenced before assignment

In [9]:
def get_prices_only(d, normalized=False, bid=None):
    """ For a given 3-tuple, return an array of prices. Strip the timestamps
    """
    ondem_price = d[1]
    if normalized:
        return d[2][:,1]/ondem_price  #time, price
    else :
        return d[2][:,1]
    
#normalized_price(data[0])

In [10]:
prices = get_prices_only(data[0])
scipy.stats.describe(prices)

DescribeResult(nobs=23459, minmax=(0.064100000000000004, 6.5), mean=0.14383601432286117, variance=0.15619643081734755, skewness=10.691686539098475, kurtosis=125.33106737538452)

In [ ]:
def convert_to_pandas(d):
    """ Called immediately after get_spot_data. Convert the price trace to dataframe """
    for (m,o,pricetrace) in d:
        df.

In [11]:
x = get_prices_only(data[0])
y = get_prices_only(data[3])

n = min(x.shape[0], y.shape[0])


In [12]:
def my_covariance(x,y):
    """
    This yields similiar results to
    pd.Series(x).cov(pd.Series(y)) """
    xbar = x.mean()
    ybar = y.mean()
    x = x-xbar
    y = y-ybar
    n = min(x.shape[0], y.shape[0])
    x = x[:n-1]
    y = y[:n-1]
    cov = (x*y)
    cov = np.sum(cov)/(n-1)
    print cov
    return cov


In [13]:
my_covariance(x,y)

-8.02333628098e-05


-8.0233362809846391e-05

In [14]:
pd.Series(x).cov(pd.Series(y))

-8.0003161504696132e-05

In [16]:

d0=data[0]
d1=data[1]
print d0

({'OS': 'Linux', 'region': 'us-east-1', 'AZ': 'a', 'type': 'g2.2xlarge'}, 0.65, array([[  1.42611080e+09,   6.52000000e-02],
       [  1.42611117e+09,   6.50000000e-02],
       [  1.42611153e+09,   6.51000000e-02],
       ..., 
       [  1.44108320e+09,   1.25300000e-01],
       [  1.44108337e+09,   1.18500000e-01],
       [  1.44108353e+09,   1.14200000e-01]]))


In [17]:
s0 = pd.Series(d0[2][:,1], index=d0[2][:,0])